# Clasificación supervisada: Árbol de Decisión vs SVM vs Random Forest vs Regresión Logística  
**Dataset:** Social Network Ads (clasificación binaria: `Purchased`)  

Este notebook implementa el flujo completo solicitado en el taller:
1. **EDA** (análisis exploratorio)
2. **Preprocesamiento** (codificación, escalado, split 80/20)
3. **Entrenamiento** de **4 modelos**:  
   - Árbol de decisión  
   - SVM (con ajuste de `kernel` y `C`)  
   - Random Forest  
   - Regresión Logística  
4. **Comparación** con métricas: accuracy, precision, recall, f1-score, matriz de confusión  
5. **Conclusiones técnicas**  


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

import matplotlib.pyplot as plt


## 1) Carga de datos

In [ ]:
df = pd.read_csv(r"/mnt/data/Social_Network_Ads.csv")
df.head()


## 2) EDA (Análisis exploratorio)

In [ ]:
df.info()


In [ ]:
# Estadística descriptiva
df.describe(include="all")


In [ ]:
# Balance de clases
class_counts = df["Purchased"].value_counts().sort_index()
class_ratio = (class_counts / class_counts.sum()).round(3)

class_counts, class_ratio


In [ ]:
# Nulos
df.isna().sum()


### Visualizaciones (matplotlib)

In [ ]:
# Histogramas de variables numéricas por clase
num_cols = ["Age", "EstimatedSalary"]

for col in num_cols:
    plt.figure()
    for y in [0, 1]:
        subset = df[df["Purchased"] == y][col]
        plt.hist(subset, bins=20, alpha=0.6, label=f"Purchased={y}")
    plt.title(f"Distribución de {col} por clase")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.legend()
    plt.show()

# Boxplots por clase
for col in num_cols:
    plt.figure()
    data0 = df[df["Purchased"]==0][col]
    data1 = df[df["Purchased"]==1][col]
    plt.boxplot([data0, data1], labels=["0", "1"])
    plt.title(f"Boxplot de {col} por Purchased")
    plt.xlabel("Purchased")
    plt.ylabel(col)
    plt.show()

# Matriz de correlación (solo numéricas)
plt.figure()
corr = df[["Age","EstimatedSalary","Purchased"]].corr()
plt.imshow(corr, aspect="auto")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar()
plt.title("Matriz de correlación (variables numéricas)")
plt.tight_layout()
plt.show()


## 3) Preprocesamiento  
**Decisiones clave (justificación):**
- `User ID` se elimina porque es un identificador y no aporta señal predictiva.
- `Gender` es categórica, se codifica con One-Hot (binaria).
- Se escala **solo para modelos sensibles a magnitud** (SVM / Regresión logística). Para simplificar, el escalado se aplica dentro del pipeline y no afecta a árboles.
- Split 80/20 con `stratify` para conservar el balance de clases.


In [ ]:
X = df.drop(columns=["Purchased"])
y = df["Purchased"]

# Eliminar ID
X = X.drop(columns=["User ID"])

cat_features = ["Gender"]
num_features = ["Age", "EstimatedSalary"]

preprocess_scaled = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(drop="if_binary", handle_unknown="ignore"), cat_features),
    ],
    remainder="drop"
)

preprocess_no_scale = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_features),
        ("cat", OneHotEncoder(drop="if_binary", handle_unknown="ignore"), cat_features),
    ],
    remainder="drop"
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


## 4) Entrenamiento de modelos

In [ ]:
# Modelo 1: Árbol de decisión
dt_model = Pipeline(steps=[
    ("prep", preprocess_no_scale),
    ("clf", DecisionTreeClassifier(random_state=42))
])

# Modelo 2: SVM (GridSearch para kernel y C)
svm_pipe = Pipeline(steps=[
    ("prep", preprocess_scaled),
    ("clf", SVC())
])

svm_param_grid = {
    "clf__kernel": ["linear", "rbf"],
    "clf__C": [0.1, 1, 10, 100],
    "clf__gamma": ["scale", "auto"]  # aplica principalmente para rbf
}

svm_model = GridSearchCV(
    svm_pipe,
    svm_param_grid,
    scoring="f1",
    cv=5,
    n_jobs=-1
)

# Modelo 3: Random Forest
rf_model = Pipeline(steps=[
    ("prep", preprocess_no_scale),
    ("clf", RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight="balanced_subsample"
    ))
])

# Modelo 4: Regresión Logística (con escalado)
lr_model = Pipeline(steps=[
    ("prep", preprocess_scaled),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        solver="lbfgs"
    ))
])

models = {
    "DecisionTree": dt_model,
    "SVM(GridSearch)": svm_model,
    "RandomForest": rf_model,
    "LogisticRegression": lr_model,
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(f"Entrenado: {name}")


## 5) Evaluación y comparación (métricas obligatorias)

In [ ]:
def eval_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    return {
        "Modelo": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "CM": cm
    }

results = []
for name, model in models.items():
    results.append(eval_model(name, model, X_test, y_test))

results


In [ ]:
# Tabla resumen
summary = pd.DataFrame([{k:v for k,v in r.items() if k!="CM"} for r in results])
summary = summary.sort_values("F1", ascending=False).reset_index(drop=True)
summary


In [ ]:
# Matrices de confusión (gráficas)
for r in results:
    cm = r["CM"]
    plt.figure()
    plt.imshow(cm, aspect="auto")
    plt.title(f"Matriz de confusión - {r['Modelo']}")
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.xticks([0,1], ["0","1"])
    plt.yticks([0,1], ["0","1"])
    for (i,j), val in np.ndenumerate(cm):
        plt.text(j, i, str(val), ha="center", va="center")
    plt.colorbar()
    plt.tight_layout()
    plt.show()


In [ ]:
# Comparación con barplot (Accuracy / Precision / Recall / F1)
metrics_cols = ["Accuracy", "Precision", "Recall", "F1"]
plt.figure()
x = np.arange(len(summary["Modelo"]))
width = 0.2

for i, m in enumerate(metrics_cols):
    plt.bar(x + i*width, summary[m], width, label=m)

plt.xticks(x + width*1.5, summary["Modelo"], rotation=20, ha="right")
plt.ylim(0, 1.05)
plt.title("Comparación de métricas por modelo")
plt.legend()
plt.tight_layout()
plt.show()


## 6) Reporte rápido (mejor modelo)

In [ ]:
best_name = summary.loc[0, "Modelo"]
best_model = models[best_name]

print("Mejor modelo por F1:", best_name)
if hasattr(best_model, "best_params_"):
    print("Mejores hiperparámetros (SVM):", best_model.best_params_)

y_pred_best = best_model.predict(X_test)
print("\nClassification report:")
print(classification_report(y_test, y_pred_best, zero_division=0))


## 7) Conclusiones técnicas (ejemplo)
- **SVM** suele beneficiarse del escalado; su desempeño depende fuerte de `C` y del kernel.
- **Regresión logística** es una buena línea base, rápida y estable; también requiere escalado.
- **Árbol de decisión** es interpretable, pero puede sobreajustar si no se limita la profundidad.
- **Random Forest** tiende a mejorar estabilidad vs un solo árbol al promediar múltiples árboles.
- La métrica **F1** es útil si quieres balancear precisión y recall, especialmente si la clase positiva es minoritaria.
